In [1]:
from pyspark.sql import SparkSession
import os

data_dir = '../../data/Spark_ML/Natural_Language_Processing/smsspamcollection/'

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

sen_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish Java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')
], ['id', 'sentence'])

sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



## Tokenizers

In [5]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

count_tokens = udf(lambda words:len(words), IntegerType())

tokenized = tokenizer.transform(sen_df)

tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [7]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [8]:
rg_tokenized = regex_tokenizer.transform(sen_df)

rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()


+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



## Stop word removal

In [10]:
from pyspark.ml.feature import StopWordsRemover

sentence_dataframe = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Mary', 'had', 'a', 'litte', 'lamb'])
], ['id', 'tokens'])

sentence_dataframe.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [11]:
stop_words_remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

remover = stop_words_remover.transform(sentence_dataframe)

remover.show()

+---+--------------------+-------------------+
| id|              tokens|           filtered|
+---+--------------------+-------------------+
|  0|[I, saw, the, gre...|[saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, litte, lamb]|
+---+--------------------+-------------------+



## n-gram extractor

In [13]:
from pyspark.ml.feature import NGram

word_df = spark.createDataFrame([
    (0, ['Hi', 'I', 'heard', 'about', 'Spark']),
    (1, ['I', 'wish', 'Java', 'could', 'use', 'case', 'classes']),
    (2, ['Logistic', 'regression', 'models', 'are', 'neat'])
], ['id', 'sentence'])

ngram = NGram(n=2, inputCol='sentence', outputCol='grams')

ngram.transform(word_df).show(truncate=False)

+---+------------------------------------------+------------------------------------------------------------------+
|id |sentence                                  |grams                                                             |
+---+------------------------------------------+------------------------------------------------------------------+
|0  |[Hi, I, heard, about, Spark]              |[Hi I, I heard, heard about, about Spark]                         |
|1  |[I, wish, Java, could, use, case, classes]|[I wish, wish Java, Java could, could use, use case, case classes]|
|2  |[Logistic, regression, models, are, neat] |[Logistic regression, regression models, models are, are neat]    |
+---+------------------------------------------+------------------------------------------------------------------+



## TF-IDF representation

In [27]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sen_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (0, 'I wish Java could use case classes'),
    (1, 'Logistic regression models are neat')
], ['label', 'sentence'])

sen_df.show()



+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|    0|Hi I heard about ...|
|    0|I wish Java could...|
|    1|Logistic regressi...|
+-----+--------------------+



In [28]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
words_df = tokenizer.transform(sen_df)

words_df.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|Hi I heard about ...|[hi, i, heard, ab...|
|    0|I wish Java could...|[i, wish, java, c...|
|    1|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [33]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures', outputCol='features')

featurized_tf = hashing_tf.transform(words_df)

featurized_tf.select(['label', 'rawFeatures']).show(truncate=False)


idf_model = idf.fit(featurized_tf)
rescaled_df = idf_model.transform(featurized_tf)

rescaled_df.select(['label', 'features']).show(truncate=False)


+-----+------------------------------------------------------------------------------------+
|label|rawFeatures                                                                         |
+-----+------------------------------------------------------------------------------------+
|0    |(262144,[18700,19036,33808,66273,173558],[1.0,1.0,1.0,1.0,1.0])                     |
|0    |(262144,[19036,20719,55551,58672,98717,109547,192310],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|1    |(262144,[46243,58267,91006,160975,190884],[1.0,1.0,1.0,1.0,1.0])                    |
+-----+------------------------------------------------------------------------------------+

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                      

In [31]:
from pyspark.ml.feature import CountVectorizer

df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ['id', 'words'])

df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [32]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

model = cv.fit(df)

result = model.transform(df)

result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

